# Regressionsanalyse


Diese Klasse behinhaltet die Methoden

- smape: liefert den smape (Symmetric mean absolute percentage error) zwischen einer Target Seite und der Predicteten Seite zu der Target Seite.
- lineareRegression: führt eine Lineare Regression für eine Target Seite und deren Backlink Seiten (Seiten, die auf diese Target Seite verlinken) aus. Hierfür werden alle Backlinks dieser Seite geholt und dann in dem Model der linearen Regression bearbeitet. Zudem wird die predictete Seite geplottet, der SMAPE zwischen Target Seite und der predicteten Seite und zudem wird auch noch der SMAPE als Boxplot dargestellt.
- logisticRegression: führt eine Logistische Regression für eine Target Seite und deren Backlink Seiten (Seiten, die auf diese Target Seite verlinken) aus. Hierfür werden alle Backlinks dieser Seite geholt und dann in dem Model der logistischen Regression bearbeitet. Zudem wird die predictete Seite geplottet, der SMAPE zwischen Target Seite und der predicteten Seite und zudem wird auch noch der SMAPE als Boxplot dargestellt.


Um eine Regression durchzuführen ist es nötig unten unter "article" die Seite anzugeben, mit der man eine Regression ausführen möchte.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pandas.core import frame
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from linkmethods import get_views, get_links

In [ ]:
def smape(target, forecast):
    if type(target) == pd.core.frame.DataFrame:
        target = target.values

    denominator = np.abs(target) + np.abs(forecast)
    flag = denominator == 0.

    smape = 2 * (
            (np.abs(target - forecast) * (1 - flag)) / (denominator + flag)
    )
    return smape


In [ ]:
def lineareRegression(article):
    mainviews = get_views(article)
    backlinksMainview = get_links(article, "linkshere")
    df = pd.DataFrame()
    len1 = len(mainviews)

    for entry in backlinksMainview:  # for every backlink from the mainpage
        views = get_views(entry)  # get the views
        len2 = len(views)
        if len2 < len1:  # then compare if they have the same amount of entries
            diff = len1 - len2
            for i in range(0, diff):
                views.append(0)

        df.insert(loc=0, column=entry, value=views)

    x = df
    y = mainviews

    model = LinearRegression()
    model.fit(x, y)

    intercept = model.intercept_
    slope = model.coef_
    r_sq = model.score(x, y)

    print("intercept:", intercept)
    print("slope:", slope)
    print("coefficient of determination:", r_sq)

    yhat = model.predict(x)
    yhat[yhat < 0] = 0

    x1 = np.linspace(2015, 2022.5, num=len(yhat))

    plt.plot(x1, yhat)
    plt.xlabel("Time")
    plt.ylabel("Views")
    plt.title(article + " Lineare Regression")
    plt.show()

    # relativer fehler
    differenceViews = smape(mainviews, yhat)
    plt.plot(x1, differenceViews)
    plt.xlabel("Difference " + article + " to all Backlinks")
    plt.ylabel("Average Error")
    plt.show()


    plt.boxplot(differenceViews)
    plt.title("Lin Regression")
    plt.ylabel("Average Error")
    plt.xlabel(article)
    plt.show()

    print("SMAPE Lin Regression: ", differenceViews.mean())

    return intercept, slope, r_sq

In [ ]:
def logisticRegression(article):
    mainviews = get_views(article)
    backlinksMainview = get_links(article, "linkshere")
    df = pd.DataFrame()
    len1 = len(mainviews)

    for entry in backlinksMainview:
        views = get_views(entry)
        len2 = len(views)
        if len2 < len1:
            diff = len1 - len2
            for i in range(0, diff):
                views.append(0)

        df.insert(loc=0, column=entry, value=views)

    x = df
    y = mainviews

    model = LogisticRegression()
    model.fit(x, y)

    intercept = model.intercept_
    slope = model.coef_
    r_sq = model.score(x, y)

    print("intercept:", intercept)
    print("slope:", slope)
    print("coefficient of determination:", r_sq)

    yhat = model.predict(x)
    yhat[yhat < 0] = 0

    x1 = np.linspace(2015, 2022.5, num=len(yhat))

    plt.plot(x1, yhat)
    plt.xlabel("Time")
    plt.ylabel("Views")
    plt.title(article + " Logarithmisch Lineare Regression")
    plt.show()

    # relativer fehler
    differenceViews = smape(mainviews, yhat)
    plt.plot(x1, differenceViews)
    plt.xlabel("Difference " + article + " to all Backlinks")
    plt.ylabel("Average Error")
    plt.show()

    plt.boxplot(smape(mainviews, yhat))
    plt.title("Log Regression")
    plt.title("Log Regression")
    plt.ylabel("Average Error")
    plt.xlabel(article)
    plt.show()

    print("SMAPE Log-Lin Regression: ", differenceViews.mean())

In [ ]:
article = "Schmetterlinge"
lineareRegression(article)
logisticRegression(article)